Load Data

In [ ]:
# !pip install yfinance

import yfinance as yf
import sys
from pathlib import Path

src_path = Path().resolve().parents[0] / "src"
sys.path.append(str(src_path))

from data_loader import load_tsla_data
tsla = load_tsla_data()

In [4]:
!pip install numpy
import numpy as np
import sys
from pathlib import Path

# Add ../src to sys.path
src_path = Path().resolve().parents[0] / "src"
sys.path.append(str(src_path))

from lstm_model import prepare_lstm_data

SEQ_LEN = 60  # You can tune this later

X_train, y_train, X_test, scaler = prepare_lstm_data(train, test, seq_length=SEQ_LEN)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ModuleNotFoundError: No module named 'tensorflow'

Build and Train the LSTM Model

In [ ]:
import sys
from pathlib import Path

# Add ../src to sys.path
src_path = Path().resolve().parents[0] / "src"
sys.path.append(str(src_path))

from lstm_model import build_lstm_model

model = build_lstm_model(input_shape=(SEQ_LEN, 1))
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)


Chronological split

In [ ]:
train = tsla[:'2023-12-31']
test = tsla['2024-01-01':]

Train ARIMA

In [ ]:
import sys
from pathlib import Path

src_path = Path().resolve().parents[0] / "src"
sys.path.append(str(src_path))

from arima_model import train_arima_model, forecast_arima

arima_model = train_arima_model(train)
forecast_series, conf_int = forecast_arima(arima_model, test)


Forecast Using LSTM

In [ ]:
import sys
from pathlib import Path

# Add ../src to sys.path
src_path = Path().resolve().parents[0] / "src"
sys.path.append(str(src_path))

from lstm_model import forecast_lstm

predicted_lstm = forecast_lstm(model, X_test, scaler)

# Wrap in a Pandas Series with the correct index
forecast_series_lstm = pd.Series(predicted_lstm, index=test.index)


Evaluate

In [ ]:
import sys
from pathlib import Path

# Add ../src to sys.path
src_path = Path().resolve().parents[0] / "src"
sys.path.append(str(src_path))

from utils import evaluate_forecast

mae, rmse, mape = evaluate_forecast(test, forecast_series)
print(f"ARIMA -> MAE: {mae:.2f}, RMSE: {rmse:.2f}, MAPE: {mape:.2f}%")

Evaluate and Visualize 

In [ ]:
import sys
from pathlib import Path

# Add ../src to sys.path
src_path = Path().resolve().parents[0] / "src"
sys.path.append(str(src_path))

from utils import evaluate_forecast

mae_lstm, rmse_lstm, mape_lstm = evaluate_forecast(test, forecast_series_lstm)
print(f"LSTM Forecast → MAE: {mae_lstm:.2f}, RMSE: {rmse_lstm:.2f}, MAPE: {mape_lstm:.2f}%")

# Plot
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(test, label='Actual')
plt.plot(forecast_series_lstm, label='LSTM Forecast')
plt.title("LSTM Forecast vs Actual TSLA Price (2024–2025)")
plt.legend()
plt.show()
